In [22]:
from Libs.functions import load_df, load_params
import os
import pandas as pd

ROOT = os.path.dirname(os.path.realpath("test.ipynb"))
PARAMS_PATH = os.path.join(ROOT, 'Bin', 'parameters.json')

params = load_params(PARAMS_PATH)
params

{'CONVERSION_RATE': 14.3,
 'CENTER_X': 327,
 'CENTER_Y': 258,
 'FPS': 30,
 'DURATION': 600,
 'EC_THRESHOLD': 3,
 'SPEED_THRESHOLD_1': 1,
 'SPEED_THRESHOLD_2': 5,
 'INTERACTION_THRESHOLD': 3,
 'FIGHTING_THRESHOLD': 3,
 'CHASING_THRESHOLD': 3}

In [23]:
import_file_path = r"D:\Code\Michael\CrayfishBehavior\Input\SRT14-C-1DLC_resnet152_Crayfish behaviorNov21shuffle1_180000_el_filtered.csv"

if import_file_path.endswith('.xlsx'):
    # check if excel file contains multiple sheets
    if len(pd.ExcelFile(import_file_path).sheet_names) > 1:
        print('Warning: Excel file contains multiple sheets, only the first sheet will be analyzed.')
    # if excel file contains multiple sheets, select the first sheet, no matter its name is
    sheet_0_name = pd.ExcelFile(import_file_path).sheet_names[0]
    df = pd.read_excel(import_file_path, sheet_name=sheet_0_name)
    # df = pd.read_excel(import_file_path)
elif import_file_path.endswith('.csv'):
    df = pd.read_csv(import_file_path)

C:\Users\ELAINE HUYNH\AppData\Local\Temp\ipykernel_23940\322075593.py:12: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(import_file_path)


In [24]:
input_df = df

In [30]:
if 'scorer' in input_df.columns:
    input_df.columns = input_df.iloc[0]
    input_df = input_df.drop(0)

#get unique values of row index 0
row_0 = input_df.iloc[0].unique()
row_0 = [x for x in row_0 if str(x) != 'bodyparts']
# ['Rostrum', 'Head', 'Telson', 'LeftPincer', 'LeftChelipod', 'RightPincer', 'RightChelipod']
new_columns = []
for x in row_0:
    new_columns.append(x+"_X")
    new_columns.append(x+"_Y")
    new_columns.append(x+"_likelihood")

length = params['DURATION'] * params['FPS'] + 1
    

['bodyparts' 'Rostrum' 'Head' 'Telson' 'LeftPincer' 'LeftChelipod'
 'RightPincer' 'RightChelipod']
['Rostrum', 'Head', 'Telson', 'LeftPincer', 'LeftChelipod', 'RightPincer', 'RightChelipod']


In [21]:
def get_CF(input_df, input_num, col1, coln):
    # Take the data of crawfish1 from df to a new df called CF1
    CF = input_df.iloc[2:,col1:coln]
    # Rename the columns
    CF.columns = new_columns
    # Remove columns with _likelihood
    CF = CF.loc[:,~CF.columns.str.contains('_likelihood')]
    # Take only 18001 rows
    CF = CF.iloc[:length+1]
    # Check if CF1 have rows with NaN
    if CF.isnull().values.any():
        print(f"CF{input_num} has NaN")
        # fill it with previous value
        CF = CF.fillna(method='ffill')
        print(f'Filled NaN in CF{input_num} with previous value')
    else:
        print(f"CF{input_num} has no NaN")
    # Reset Index
    CF = CF.reset_index(drop=True)

    # Change all values in CF1 and CF2 to float
    CF = CF.astype(float)

    return CF

CF1 = get_CF(input_df, 1, 1, len(row_0)*3+1)
CF1

ValueError: Length mismatch: Expected axis has 42 elements, new values have 87 elements